In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'philschmid/flan-t5-base-samsum'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/home/alic/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alic/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
sum(p.numel() for p in model.parameters())

406290432

In [3]:
tokenizer.model_max_length

1024

In [4]:
tokenizer.max_len_single_sentence

1022

In [5]:
tokenizer.num_special_tokens_to_add()

2

In [6]:
with open('../TextExtractor/files/P-1005/20001023-0594_3203938.txt', 'r') as f:
    txt = f.read().strip()

In [7]:
len(txt)

9468

In [8]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/alic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/alic/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
sentences = nltk.tokenize.sent_tokenize(txt)
len(sentences)

51

In [10]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

240

In [11]:
# chunking
length = 0
chunk = ''
all_chunks = []
counter = -1

for sentence in sentences:
    counter += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length

    if combined_length <= tokenizer.max_len_single_sentence:
        chunk += sentence + ' '
        length = combined_length

        if counter == len(sentences) - 1:
            all_chunks.append(chunk.strip())
    else:
        all_chunks.append(chunk.strip())
        length = 0
        chunk = ''

        chunk += sentence + ' '
        length = len(tokenizer.tokenize(sentence))

len(all_chunks)

4

In [12]:
[len(tokenizer.tokenize(c)) for c in all_chunks]

[990, 994, 932, 217]

In [13]:
[len(tokenizer(c).input_ids) for c in all_chunks]

[992, 996, 934, 219]

In [14]:
sum([len(tokenizer.tokenize(c)) for c in all_chunks])

3133

In [15]:
len(tokenizer.tokenize(txt))

Token indices sequence length is longer than the specified maximum sequence length for this model (3590 > 1024). Running this sequence through the model will result in indexing errors


3590

In [16]:
chunk_summary_inputs = [tokenizer(chunk, return_tensors="pt") for chunk in all_chunks]

In [17]:
chunk_summaries = []

for input in chunk_summary_inputs:
    output = model.generate(**input)
    summary = tokenizer.decode(*output, skip_special_tokens=True)
    chunk_summaries.append(summary)

In [24]:
chunk_summaries

["Public Service Company of Colorado wants to transfer the license for the Boulder Canyon Project No. 1005 1/ to the City of Boulder, Colorado. The project is located on Middle Boulder Creek in Boulder County, Colorado, in Roosevelt National Forest. The transfer is sought pursuant to an asset purchase agreement, dated February 29, 2000, as amended June 16, 2000. Trout Unlimited's motion to intervene was granted.",
 "Boulder will file for Commission approval such revised project exhibits as may be necessary to accurately reflect the retained property rights of Project No. 1005-006. Boating on Barker, LLC expresses an interest in increased boating activity on the project's Barker Reservoir. USDA is concerned about the protection of riparian areas, threatened and endangered species, and other resources within the project boundary.",
 'Boulder is qualified to hold the license and to operate the properties under the license. The transfer of the license for Project No. 1005 from Public Servi